In [3]:
import requests

In [4]:
import geopandas as gpd
import pandas as pd

## Open State data

In [14]:
file_path = "data/cb_2018_us_state_500k/cb_2018_us_state_500k.shp"
state_gdf = gpd.read_file(file_path)

## Get Columm Data

In [15]:
r = requests.get("https://api.census.gov/data/2023/acs/acs5/subject/variables.json")
columns_obj = r.json()

In [16]:
columns_to_replace = {
    key: val["label"]
    for key, val in columns_obj["variables"].items()
    if key.startswith("S0802")
}

In [17]:
columns_obj["variables"]["S0802_C05_001E"]

{'label': 'Estimate!!Worked from home!!Workers 16 years and over',
 'concept': 'Means of Transportation to Work by Selected Characteristics',
 'predicateType': 'int',
 'group': 'S0802',
 'limit': 0,
 'attributes': 'S0802_C05_001EA,S0802_C05_001M,S0802_C05_001MA'}

In [18]:
columns_obj["variables"]["S0802_C01_001E"]

{'label': 'Estimate!!Total!!Workers 16 years and over',
 'concept': 'Means of Transportation to Work by Selected Characteristics',
 'predicateType': 'int',
 'group': 'S0802',
 'limit': 0,
 'attributes': 'S0802_C01_001EA,S0802_C01_001M,S0802_C01_001MA'}

### Get columns to query and rename for later

In [19]:
columns = ["GEO_ID", "S0802_C01_001E", "S0802_C05_001E", "S0802_C05_001M"]
columns_formatted = ",".join(columns)

In [20]:
response = requests.get(
    f"https://api.census.gov/data/2023/acs/acs5/subject?get={columns_formatted}&for=state:*"
)

In [21]:
data = response.json()
columns = data[0]
rows = data[1:]
commuting_df = pd.DataFrame(rows, columns=columns)

In [22]:
commuting_df = commuting_df[columns]
commuting_df[columns[1:]] = commuting_df[columns[1:]].astype(int)

In [23]:
columns

['GEO_ID', 'S0802_C01_001E', 'S0802_C05_001E', 'S0802_C05_001M', 'state']

In [24]:
def check_margin_error(row) -> str:
    wfh_col = row["S0802_C05_001E"]
    wfh_moe = row["S0802_C05_001M"]
    workers_total = row["S0802_C01_001E"]

    if not wfh_col:
        return None

    rmoe_val = abs(wfh_moe / wfh_col)
    if rmoe_val < 0.50:
        return round((wfh_col / workers_total) * 100, 1)
    else:
        return None

In [25]:
commuting_df["PERCENT"] = commuting_df.apply(
    lambda row: check_margin_error(row), axis=1
)

In [28]:
columns_to_replace["GEO_ID"] = "AFFGEOID"
commuting_df = commuting_df.rename(columns=columns_to_replace)

In [29]:
commuting_df.columns

Index(['AFFGEOID', 'Estimate!!Total!!Workers 16 years and over',
       'Estimate!!Worked from home!!Workers 16 years and over',
       'S0802_C05_001M', 'state', 'PERCENT'],
      dtype='object')

## Merge Data

In [33]:
commuting_gdf = state_gdf.merge(commuting_df, on="AFFGEOID", how="inner")

In [35]:
commuting_gdf = commuting_gdf.to_crs(9311)
commuting_gdf.to_file("data/Work_From_Home_Per_State.gpkg")